In [ ]:
import numpy as np
import tifffile as tiff
from pathlib import Path
import flammkuchen as fl
import os.path
import matplotlib.pyplot as plt
import json

In [ ]:
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files_path = list(master.glob("*_f*/*f*/anatomy_affine_res_2avg2.tif"))
num_fish = len(files_path)
print(num_fish)

In [ ]:
def get_xform_values(xform_path):
    with open(xform_path) as f:
        lines = f.readlines()
    lines = lines[6:11]
    
    trans_values = np.zeros((5, 3))
    for i in range(5):
        tmp_line = lines[i]
        tmp_line = tmp_line[8:-2]
        tmp_line = tmp_line.split()
        tmp_line = [float(j) for j in tmp_line]
        trans_values[i] = tmp_line
        
    return trans_values

In [ ]:
def values2matrix(trasformation_vals):
    
    # translation:
    m_trans = np.identity(4)
    for i in range(3):
        m_trans[i,3] = trasformation_vals[0,i]
    
    # rotation:
    # in cmtk the rotation values are ordered: x, y, z
    m_rot_z = np.identity(4)
    theta = trasformation_vals[1, 2]
    m_rot_z[0,0] = np.cos(theta)
    m_rot_z[0,1] = -1 * np.sin(theta)
    m_rot_z[1,0] = np.sin(theta)
    m_rot_z[1,1] = np.cos(theta)
    
    m_rot_x = np.identity(4)
    theta = trasformation_vals[1, 0]
    m_rot_x[1,1] = np.cos(theta)
    m_rot_x[1,2] = -1 * np.sin(theta)
    m_rot_x[2,1] = np.sin(theta)
    m_rot_x[2,2] = np.cos(theta)
    
    m_rot_y = np.identity(4)
    theta = trasformation_vals[1, 1]
    m_rot_y[0,0] = np.cos(theta)
    m_rot_y[2,0] = -1 * np.sin(theta)
    m_rot_y[0,2] = np.sin(theta)
    m_rot_y[2,2] = np.cos(theta)
    
    # scale: 
    m_scale = np.identity(4)
    for i in range(3):
        m_scale[i,i] = trasformation_vals[2,i]
    
    # shear: 
    # in cmtk the rotation values are ordered: x, y, z. I assume shear values are the same 
    m_shear = np.identity(4)
    m_shear[0,1] = trasformation_vals[3,0]
    m_shear[0,2] = trasformation_vals[3,0]
    m_shear[1,0] = trasformation_vals[3,1]
    m_shear[1,2] = trasformation_vals[3,1]
    m_shear[2,0] = trasformation_vals[3,2]
    m_shear[2,1] = trasformation_vals[3,2]
    
    return m_shear @ m_scale @ m_rot_z @ m_rot_y @ m_rot_x @ m_trans


In [ ]:
fish_path = Path(os.path.dirname(str(files_path[0])))
suite2p_file = fl.load(fish_path / "data_from_suite2p_unfiltered.h5")
suite2p_file.keys()
coords_orig = suite2p_file['coords']
coords = np.ones((np.shape(coords_orig)[0], 4))
coords[:, 0:3] = coords_orig
num_planes = int(np.max(coords[:,0]) + 1)
print(num_planes)

In [ ]:
# For a single fish:
# Getting the coordinates: 
fish_path = Path(os.path.dirname(str(files_path[0])))
suite2p_file = fl.load(fish_path / "data_from_suite2p_unfiltered.h5")
coords_orig = suite2p_file['coords']
coords = np.ones((np.shape(coords_orig)[0], 4))
coords[:, 0:3] = coords_orig
stack_shape = np.shape(suite2p_file['anatomy_stack'])

# Getting the resolution values to scale the coordinates:
with open(next(Path(fish_path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
n_planes = lsconfig["triggering"]["n_planes"]
res = [z_tot_span / n_planes, 0.6, 0.6]
for i in range(3):
    coords[:, i] *= res[i]
    
# Getting the transformation values:
xform1_path = fish_path / "anatomy_affine_res_2avg.xform" / "registration"
xform2_path = fish_path / "anatomy_affine_res_2avg2.xform" / "registration"
trans_values1 = get_xform_values(xform1_path)
trans_values2 = get_xform_values(xform2_path)

center_vals1 = trans_values2[4]
center_vals2 = trans_values2[4]

In [ ]:
print(trans_values2)

In [ ]:
# Transforming the coordinates: 
t_matrix1 = values2matrix(trans_values1)
t_matrix2 = values2matrix(trans_values2)

num_coords = np.shape(coords)[0]
t_coords = np.ones_like(coords)
# Removing center values: 
for i in range(num_coords):
    t_coords[i,0:3] = coords[i,0:3] - center_vals1
        
t_coords = t_matrix1 @ t_coords.T
t_coords = t_coords.T

for i in range(num_coords):
    t_coords[i,0:3] = t_coords[i,0:3] + center_vals1
    
# Removing center values: 
for i in range(num_coords):
    t_coords[i,0:3] = t_coords[i,0:3] - center_vals2
        
t_coords = t_matrix2 @ t_coords.T
t_coords = t_coords.T

for i in range(num_coords):
    t_coords[i,0:3] = t_coords[i,0:3] + center_vals2


In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15,10))
num_cells = np.shape(coords)[1]
title_list = ['Horizontal', 'Lateral', 'Frontal']
for i in range(3):
    ax[0,i].axis('off')
    ax[0,i].set_title(title_list[i])
    ax[1,i].axis('off')
ax[0,0].scatter(coords[:,1], coords[:,2], s=10)
ax[1,0].scatter(t_coords[:,1], t_coords[:,2], s=10)

ax[0,1].scatter(coords[:,2], coords[:,0], s=10)
ax[1,1].scatter(t_coords[:,2], t_coords[:,0], s=10)

ax[0,2].scatter(coords[:,1], coords[:,0], s=10)
ax[1,2].scatter(t_coords[:,1], t_coords[:,0], s=10)
plt.show()
fig.savefig(fish_path / "transformed_coords.jpg")

In [ ]:
# For all fish: 
for i in files_path:
    # Getting the coordinates: 
    fish_path = Path(os.path.dirname(str(i)))
    suite2p_file = fl.load(fish_path / "data_from_suite2p_unfiltered.h5")
    coords_orig = suite2p_file['coords']
    coords_orig = suite2p_file['coords']
    coords = np.ones((np.shape(coords_orig)[0], 4))
    coords[:, 0:3] = coords_orig
    
    # Getting the resolution values to scale the coordinates:
    with open(next(Path(fish_path).glob("*metadata.json")), "r") as f:
            metadata = json.load(f)
    lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
    z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
    n_planes = lsconfig["triggering"]["n_planes"]
    res = [z_tot_span / n_planes, 0.6, 0.6]
    for i in range(3):
        coords[:, i] *= res[i]
    
    # Getting the transformation values:
    xform1_path = fish_path / "anatomy_affine_res_2avg.xform" / "registration"
    xform2_path = fish_path / "anatomy_affine_res_2avg2.xform" / "registration"

    trans_values1 = get_xform_values(xform1_path)
    trans_values2 = get_xform_values(xform2_path)
    
    # Transforming the coordinates: 
    t_matrix1 = values2matrix(trans_values1)
    t_matrix2 = values2matrix(trans_values2)

    num_coords = np.shape(coords)[0]
    t_coords = np.ones_like(coords)
    # Removing center values: 
    for i in range(num_coords):
        t_coords[i,0:3] = coords[i,0:3] - center_vals1

    t_coords = t_matrix1 @ t_coords.T
    t_coords = t_coords.T

    for i in range(num_coords):
        t_coords[i,0:3] = t_coords[i,0:3] + center_vals1

    # Removing center values: 
    for i in range(num_coords):
        t_coords[i,0:3] = t_coords[i,0:3] - center_vals2

    t_coords = t_matrix2 @ t_coords.T
    t_coords = t_coords.T

    for i in range(num_coords):
        t_coords[i,0:3] = t_coords[i,0:3] + center_vals2